# Women Participation in National Parliaments - Rank as of 1st September of 2021

In [2]:
import pandas as pd, numpy as np
import plotly.express as px

In [23]:
# read data
df_historic = pd.read_csv('./data/women_in_parliament-historical_database-1945_to_2018_cleaned.csv')
df_regions = df_historic[['country','region']]
df_regions = df_regions.drop_duplicates()

df_2021 = pd.read_csv('./data/women_percent_as_of2021_cleaned.csv')
df_2021.rename(columns={'Country':'country'},inplace=True)

df_regions.set_index(['country'], inplace=True)
df_2021.set_index(['country'], inplace=True)

df_2021_regions = df_2021.join(df_regions).reset_index()

to_replace = dict({'Azerbaijan':'ASIA',
'Bangladesh':'ASIA',
'Bahrain':'MENA',
'Bahamas': 'AME',
'Bolivia (Plurinational State of)':'AME',
'Central African Republic':'SUB-SAHARAN',
'Chad':'SUB-SAHARAN',
'Chile':'AME',
'China':'ASIA',
'Congo':'SUB-SAHARAN',
"Côte d'Ivoire":'SUB-SAHARAN',
'Qatar':'MENA',
'North Macedonia': 'EUR',
'Venezuela (Bolivarian Republic of)':'AME' })

for i in range(len(df_2021_regions)):
    country  = df_2021_regions['country'].iloc[i]
    if country in to_replace.keys():
        df_2021_regions['region'].iloc[i] = to_replace[country]

#df_2021_regions.to_csv('./data/women_percent_as_of2021_with_regions.csv')

## Participation in 2021 by Region

In [25]:
fig = px.treemap(df_2021_regions, 
                 path=[px.Constant("world"), 'region', 'country'], 
                 values='%W',
                 color='%W', 
                 title='Women Representation in Parliament 2021',
                 color_continuous_scale='RdBu',
                 color_continuous_midpoint=np.average(df_2021_regions['%W'], weights=df_2021_regions['%W']),
                 hover_data = ['%W'])
fig.update_traces(root_color="red")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.data[0].hovertemplate = '%{label}<br>%{value}'
fig.show()

In [34]:
import pycountry
import pandas as pd

def get_iso3(country_name):
    try:
        iso3 = pycountry.countries.get(name=country_name).alpha_3
    except:
        iso3 = 'not_found'
    
    return iso3

# read data of Women's representation in parliament as of 2021
df21 = pd.read_csv('./data/women_percent_as_of2021.csv',skiprows=5)
# get country ISO code
df21['iso3'] = df21['Country'].apply(get_iso3)

# fix missing country codes
missing_codes = df21[df21['iso3'].str.contains('not_found')]['Country'].unique()
print(missing_codes)

bol_index = df21[df21['Country'].str.contains('Bolivia')].index[0]
df21.iloc[bol_index]['iso3']='BOL'

mda_index = df21[df21['Country'].str.contains('Moldova')].index[0]
df21.iloc[mda_index]['iso3']='MDA'

tza_index = df21[df21['Country'].str.contains('Tanzania')].index[0]
df21.iloc[tza_index]['iso3']='TZA'

usa_index = df21[df21['Country'].str.contains('States of America')].index[0]
df21.iloc[usa_index]['iso3']='USA'

kor_index = df21[df21['Country'].str.contains('Republic of Korea')].index[0]
df21.iloc[kor_index]['iso3']='KOR'

prk_index = df21[df21['Country'].str.contains("Democratic People's Republic of Korea")].index[0]
df21.iloc[prk_index]['iso3']='PRK'

cod_index = df21[df21['Country'].str.contains("Democratic Republic of the Congo")].index[0]
df21.iloc[cod_index]['iso3']='COD'

bgm_index = df21[df21['Country']=="Gambia (The)"].index[0]
df21.iloc[bgm_index]['iso3']='GBM'

irn_index = df21[df21['Country']=="Iran (Islamic Republic of)"].index[0]
df21.iloc[irn_index]['iso3']='IRN'

fsm_index = df21[df21['Country']=="Micronesia (Federated States of)"].index[0]
df21.iloc[fsm_index]['iso3']='FSM'

ven_index = df21[df21['Country']=="Venezuela (Bolivarian Republic of)"].index[0]
df21.iloc[ven_index]['iso3']='VEN'

ch_index = df21[df21['Country']=="Czech Republic"].index[0]
df21.iloc[ch_index]['iso3']='CZE'

df21 = df21.loc[:186]
df21['%W'] = df21['%W'].iloc[:186].astype('float')

df21['%W_str'] = df21['%W'].apply(lambda x:str(x)+'%')

# df21.to_csv('./data/women_percent_as_of2021_cleaned.csv',index=False)

['Bolivia (Plurinational State of)' 'Republic of Moldova'
 'United Republic of Tanzania' 'United States of America' 'Czech Republic'
 'Venezuela (Bolivarian Republic of)' 'Republic of Korea'
 "Democratic People's Republic of Korea"
 'Democratic Republic of the Congo' 'Gambia (The)'
 'Iran (Islamic Republic of)' 'Micronesia (Federated States of)']


In [36]:
hover_data = {key:False for key in df21.columns}
hover_data['%W_str']=True
fig = px.choropleth(df21, locations="iso3",color="%W", 
                    hover_name="Country",
                    hover_data= hover_data,
                    labels={'%W_str':'Perc '},
                    range_color=[10,55],  
                    color_continuous_scale='Viridis',
                    title="Women's Representation in Parliament as of 2021 (%)"                           )

fig.update_layout(margin={"r":40,"t":40,"l":40,"b":40},  autosize=False, width=1300, height=700)
fig.update_layout(geo=dict(bgcolor= 'rgba(229,209,197,180)'))
fig.update_layout(
       coloraxis_colorbar=dict(
              thicknessmode="pixels", thickness=30,
              lenmode='pixels',len=600,
              yanchor='top',y=1, x=1,
              ticks="outside", ticksuffix=" %",dtick=10
              ),
       hoverlabel=dict(
              bgcolor="#e6ffff",
              bordercolor='grey',
              font_size=12,
              font_family="Courier New",
              

              )
       )

fig.show()

In [ ]:
# If you need to save this file as a standalone html file:
#fig.write_html("./choropleth-map-plotly-python.html")